In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def minmaxscaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / denominator           
    # noise term prevents the zero division:  (denominator + 1e-7)

In [3]:
from sklearn.model_selection import train_test_split
xy = np.loadtxt('white.csv', skiprows=1, delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
ymax, ymin = np.max(y_data), np.min(y_data)
xy = minmaxscaler(xy)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=100)
y_test = y_test*(ymax-ymin)+ymin

In [4]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [5]:
## Build graph using tensorflow operations
n_hidden_1 = 36
n_input = X_train.shape[1]
keep_prob = tf.placeholder("float")
X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_input]))
}

B = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([1]))
}

In [6]:
# Linear regression
mu = multilayer_perceptron(X, W, B, keep_prob)
cost = tf.reduce_mean(tf.square(mu-Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)   

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(10001):
    cost_val, mu_val, _ = sess.run([cost, mu, train], feed_dict={X: X_train, Y: y_train, keep_prob: 0.8})
    if step % 1000 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", mu_val)

0 Cost:  22.759594 
Prediction:
 [[ -7.362415    -5.3819265   -2.3327518  ...  -4.7501564   -4.442048
    3.9758382 ]
 [ -7.55497     -3.4059973   -0.8726237  ...  -4.4447002   -2.173253
    5.544345  ]
 [ -4.411107    -2.9172115   -0.8745765  ...  -1.5355506   -2.6945672
    1.3579098 ]
 ...
 [-10.393974    -4.210293    -3.8012896  ...  -5.0768943   -0.39396995
    5.6162148 ]
 [ -7.1807427   -5.4631534   -2.1464233  ...  -3.4374495   -4.0057387
    1.7472948 ]
 [ -6.1598697   -7.205911    -0.80066913 ...  -3.6934876   -5.369325
    2.5024471 ]]
1000 Cost:  0.8784374 
Prediction:
 [[ 0.28405547  0.7276292   0.7648704  ...  0.6427522   0.6148995
   0.80910313]
 [ 0.4755441   1.4294255   1.4398189  ...  0.5720135   0.77096033
   2.1655161 ]
 [ 0.70386755 -0.20324934 -1.0311948  ... -0.7189744   0.7501955
   0.02781403]
 ...
 [ 0.8321917   1.2669861   0.4651562  ...  1.1508176   1.9760158
   1.1731286 ]
 [-0.50966334 -0.3701135   0.27835083 ...  0.05268943  0.00959957
   0.6132598 ]
 [-0

In [8]:
#표준화된 데이터의 weights, bias 추정 
print("Weight\n",sess.run(W), "\nbias:",sess.run(B))   

Weight
 {'h1': array([[-2.14120120e-01, -1.51959658e-01, -2.44084930e+00,
        -1.01031291e+00,  2.08309010e-01,  1.48246539e+00,
         1.45913169e-01, -4.87447917e-01,  4.15601283e-01,
        -6.17084444e-01, -2.44811237e-01, -3.37246835e-01,
         7.69143179e-02, -6.86997652e-01,  8.66440356e-01,
         1.84144884e-01,  4.19596285e-01, -1.93425715e+00,
         1.04067528e+00,  2.21057117e-01, -1.43311512e+00,
         6.33560196e-02, -3.07902575e-01, -7.23167241e-01,
        -1.14729667e+00, -2.38195300e+00,  1.19005024e+00,
        -5.62678754e-01,  5.40408671e-01,  1.35115087e+00,
        -1.46658003e+00,  3.93339455e-01, -1.01578999e+00,
        -1.34838402e+00,  1.13507666e-01, -4.66061503e-01],
       [ 1.72592163e-01,  7.78566718e-01,  7.40837604e-02,
         9.16263282e-01, -8.16845536e-01,  9.49536204e-01,
        -5.98771214e-01, -8.92031133e-01,  1.52148283e+00,
        -2.55658895e-01, -4.11177069e-01, -1.23596644e+00,
        -3.31412591e-02, -1.12330413e+00

In [10]:
# 예측 & MSE
muhat = sess.run(mu, feed_dict={X: X_test, keep_prob: 1.0})
pred = muhat*(ymax-ymin)+ymin
print(sess.run(tf.reduce_mean(tf.square(pred-y_test))))

0.5379652
